# Neural Network that solves the Heat Equation
 The objective of this notebook is to create a NN that solves the heat equation with a certain initial value and boundary conditions in an interval $[-1,1]$ for a time interval $[0,T]$.

 * Define the network class with a loss function that solves the heat equation. leave room for initial condition and boundary condition loss terms?
 * Write functions that train and validate the network.
 * Write functions that plot the solution and the loss over the pochs
 * Define the training data in the domain.
 * Instantiate and train a baseline network. Validate it.
 * cross val with different architectures

In [2]:
# Torch imports
import torch

import torch.autograd as autograd # computation graph
from torch import Tensor # tensor node in the computation graph
import torch.nn as nn # Neural networks
import torch.optim as optiom # Optimizer, gradient descen adam, etc.

# Other Imports
import numpy as np 
import matplotlib.pyplot as plt


## Problem definition
We want to solve the heat equation in an interval of $\mathbb{R}$
$$u_t(x,t) = u_{xx}(x,t) \qquad (x,t) \in [-1,1]\times [0,4]$$
$$u(x,0) = u_0(x) \qquad\forall x\in [-1,1]  $$
and a certain boundary condition.

We proprose a Physics Informed Neural Network $\psi_{NN}(x,t)$ and a loss function composed of 4 terms.

* $L_1 = ||\psi_{NN}(x,0)  - u_0 (x) ||_2$
* $L_2 = ||\psi_{NN}(-1,t)  - 0||_2$
* $L_3 = ||\psi_{NN}(1,t)  - 0||_2$
* $L_4 = ||\psi_{NN}(x,t)_t - \psi_{NN}(x,t)_{xx}||_2$

So our Loss function will be
$$L(\psi) := L_1(\psi) + L_2(\psi) + L_3(\psi) + L_4(\psi)$$

Of course since we can't compute exactly these $L^2$ norms, we will aproximate them with sums over some points of the domain

* $L_1 = \frac{1}{n_1}  \sum_{k=1}^{n_1} | \psi_{NN}(x_k,0)  - u_0 (x_k)|^2 $
* $L_2 = \frac{1}{n_2}  \sum_{k=1}^{n_2} | \psi_{NN}(-1,t_k) |^2 $
* $L_2 = \frac{1}{n_3}  \sum_{k=1}^{n_3} | \psi_{NN}(1,t_k) |^2 $
* $L_4 = \frac{1}{N_4}  \sum_{i=1}^{n_4} | \psi_{NN,t}((x,t)_i) - \psi_{NN,xx} ((x,t)_i) |^2 $

For some data points
* $\{x_k \}_{k=1}^{n_1}$ is a partition of $[-1,1]$. We may choose it randomly or just take a grid with a fixed step.
* Same for $\{t_k \}_{k=1}^{n_2}$ in $[0,T]$
* $\{(x,t)_i \}_{i=1}^{n_4}$ are the collocation points inside the domain that we choose randomly



## Data

We will create all of the points we will need, including a small validation data set

In [44]:
# We define our domain as $[-1,1] x [0,4]$
xm = -1
xM = 1
t0 = 0 # t siempre empieza en cero
T = 4

number_of_points = 1000

# (X, T)

#Initial condition
x_init_train = torch.linspace(xm,xM,number_of_points)
y_init_train = np.exp(-x_init_train**2 )- 1/np.e

# Reshape x
x_init_train = torch.reshape(x_init_train, (number_of_points, 1))
t_zeros = torch.zeros(number_of_points)
t_zeros = torch.reshape(t_zeros, (number_of_points,1))
x_init_train = torch.cat((x_init_train,t_zeros),dim = 1)

# Reshape y
y_init_train = torch.reshape(y_init_train, (number_of_points,1))

#Boundary condition
t_bc= torch.linspace(t0,T,number_of_points)
t_bc = torch.reshape(t_bc, (number_of_points,1))

x_bc = torch.ones(number_of_points)
x_bc = torch.reshape(x_bc,(number_of_points,1))

t_bc1_train = torch.cat((-1*x_bc,t_bc), dim = 1)
t_bc2_train = torch.cat((x_bc, t_bc), dim = 1)



In [55]:
# Colocation points
number_of_points = 10000

x_coloc_train = xm + (xM-xm)*torch.rand(number_of_points)
t_coloc_train = T*torch.rand(number_of_points)

x_coloc_train = torch.reshape(x_coloc_train,(number_of_points,1))
t_coloc_train = torch.reshape(t_coloc_train,(number_of_points,1))

X_coloc_train = torch.cat((x_coloc_train,t_coloc_train), dim = 1)
#X_coloc_train[0:10], X_coloc_train.shape


Tengo mis cuatro conjuntos de puntos para evaluar la loss

* x_init_train
* y_init_train
* t_bc1_train
* t_bc2_train
* X_coloc_train

No hize los vectores de ceros como los y de la boundary ocnidtion, pero ese total lo puedo ahcer despues

Me gustaria funcionalizar la manera como hago estos puntos.


In [24]:
x_init_train = torch.linspace(xm,xM,10)
x_init_train = torch.reshape(x_init_train, (10,1))
t_init = torch.zeros(10)

In [27]:
t_init = torch.reshape(t_init,(10,1))
t_init

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [37]:
x_train = torch.cat([x_init_train, t_init], dim = 1)
x_train.shape

torch.Size([10, 2])